# 텐서플로 2.0 시작하기: 초보자용

이 문서는 [구글 코랩](https://colab.research.google.com/notebooks/welcome.ipynb)(Colaboratory) 노트북 파일을 기반으로 2021YEHS Study ML을 위해 수정되었습니다. 파이썬 프로그램을 브라우저에서 직접 실행할 수 있기 때문에 텐서플로를 배우고 사용하기 좋은 도구입니다.

1. 파이썬 런타임(runtime)에 연결하세요: 메뉴 막대의 오른쪽 상단에서 *CONNECT*를 선택하세요.
2. 노트북의 모든 코드 셀(cell)을 실행하세요: *Runtime* > *Run all*을 선택하세요.

더 많은 예제와 자세한 안내는 [텐서플로 튜토리얼](https://www.tensorflow.org/tutorials/)을 참고하세요.

먼저 프로그램에 텐서플로 라이브러리를 임포트합니다:

In [2]:
#!pip install -q tensorflow-gpu==2.4.0-rc1
import tensorflow as tf

[MNIST 데이터셋](http://yann.lecun.com/exdb/mnist/)중 숫자 필기체 데이터를 로드하여 준비합니다. 샘플 값을 정수에서 부동소수로 변환합니다:
* Modified subset of NIST (National Institute of Standards and Technology) 약자이다. NIST에서 수집된 DB를 약간 수정한 DB
* MNIST데이터에는 숫자 필기체 뿐만 아니라 아래와 같이 의상 데이터도 함께 있다.

In [3]:
# 필기체 MNIST Data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
class_names = ['0','1','2','3','4','5','6','7','8','9']

11493376/11490434 [==============================] - ETA:  - 2s 0us/step


In [4]:
# 의상 MNIST Data
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

4423680/4422102 [==============================] - 0s 0us/step


불러온 데이터를 확인해봅시다. 이미지는 28x28 크기의 넘파이 배열이고 픽셀 값은 0과 255 사이입니다. 레이블(label)은 0에서 9까지의 정수 배열입니다. 

In [5]:
print(train_images.shape)
print(train_images[0])
print(train_labels.shape)
print(train_labels)

import matplotlib.pyplot as plt
import random
n = random.randint(0, 60000)
plt.figure()
plt.imshow(train_images[n])
plt.suptitle(class_names[train_labels[n]])
plt.colorbar()
plt.grid(False)
plt.show()

(60000, 28, 28)
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0
    0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62
   54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134
  144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178
  107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216
  216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   0   0   0   0   0   1   1   

ModuleNotFoundError: No module named 'matplotlib'

이 값은 이미지에 있는 숫자를 나타냅니다. 신경망 모델에 주입하기 전에 이 값의 범위를 0~1 사이로 조정하겠습니다. 이렇게 하려면 255로 나누어야 합니다. 훈련 세트와 테스트 세트를 동일한 방식으로 전처리하는 것이 중요합니다:

In [ ]:
train_images, test_images = train_images / 255.0, test_images / 255.0

plt.imshow(train_images[n], cmap=plt.cm.binary)
plt.suptitle(class_names[train_labels[n]])
plt.colorbar()
plt.grid(False)
plt.show()

층을 차례대로 쌓아 `tf.keras.Sequential` 모델을 만듭니다. 훈련에 사용할 옵티마이저(optimizer)와 손실 함수를 선택합니다. 
* 이 부분에서는 데이터를 매개변수로 받지 않습니다. 모델의 구조만을 명시합니다.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)), # 28x28 Matrix to 784x1 vector
  tf.keras.layers.Dense(128, activation='relu'),  
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax') # 784x1 vector to 10x1 vector
])

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

모델을 훈련하고 평가합니다:

In [ ]:
history = model.fit(train_images, train_labels, epochs=5)
model.evaluate(test_images, test_labels, verbose=1)

In [ ]:
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.subplot(1,2,2)
plt.plot(history.history['loss'])

훈련된 이미지 분류기는 이 데이터셋에서 90% 이상의 정확도를 달성합니다. 아래 함수를 통해 모델을 시각화할 수 있습니다.

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)

이제 모델이 완성되었습니다! 모델을 직접 사용해봅시다.

In [ ]:
import numpy as np
predictions = model.predict(test_images)

m = random.randint(0, np.shape(test_images)[0])
print('['+str(m) + 'th test image'+']')
plt.suptitle('['+str(m)+'th test image'+']-'+'[True = '+str(class_names[train_labels[m]])+ ' Predict: '+str(class_names[np.argmax(predictions[m])])+']')
plt.subplot(1,2,1)
plt.imshow(test_images[m], cmap=plt.cm.binary)
plt.subplot(1,2,2)
plt.bar(np.arange(10), predictions[m])
plt.xticks([i * 2 + 0.5 for i in range(len(class_names))],class_names)
plt.show()

print(predictions[m])
print(np.argmax(predictions[m]))

[불안정] TensorFlow는 [TensorBoard](https://www.tensorflow.org/tensorboard?hl=ko)라는 강력한 시각화 도구를 제공합니다. 

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(train_images, train_labels, 
          epochs=100, 
          callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir logs/fit